<h1> Libraries & Environment Setup </h1>

In [1]:
import os  # Environment management
import numpy as np  # Numpy
import random  # Determinism
import torch  # PyTorch
import torch.nn as nn  # Neural network module
import torch.optim as optim  # Optimizers
import matplotlib.pyplot as plt  # Plotting
import matplotlib.animation as animation # Animation
import warnings  # Silence some sklearn warnings
import optuna # Hyperparameter optimization

from collections import Counter  # Counting
from torchvision import datasets, transforms  # Datasets and transformations
from torch.utils.data import DataLoader, random_split
from sklearn.cluster import KMeans  # KMeans clustering algorithm
from sklearn.model_selection import train_test_split  # Train-test split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Confusion matrix
from sklearn.manifold import TSNE  # t-SNE
from tqdm.notebook import tqdm  # Progress bars

from models import CNN  # Neural network class
from trainers import KMeansConsistencyTrainer, SimCLRTrainer  # Training function
from datasets import get_mnist_loaders, split_dataset  # Dataset functions

warnings.filterwarnings("ignore")    # Silence some annoying sklearn warnings
os.environ["OMP_NUM_THREADS"] = "1"  # Due to sklearn bug

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set seeds for determinism
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Using device: cuda


<h1> MNIST Data Preparation </h1>

In [2]:
# Load MNIST dataset
train_loader, test_loader = get_mnist_loaders(batch_size=128)

# Split dataset into labeled and unlabeled subsets
mnist_train = train_loader.dataset
# labeled_data, unlabeled_data = split_dataset(mnist_train, num_labeled=250)

# Create data loaders for labeled and unlabeled data
# labeled_loader = torch.utils.data.DataLoader(labeled_data, batch_size=128, shuffle=True, num_workers=0)
# unlabeled_loader = torch.utils.data.DataLoader(unlabeled_data, batch_size=128, shuffle=True, num_workers=0)

<h1> Training Model </h1>

In [3]:
# Experiment setup
def experiment(values_of_M, epochs=10, evaluate_every=1, lambda_kmeans=0.1, lambda_consistency=0.1, use_consistency=False, use_unlabeled=True, save_dir="tsne_images", output_file="tsne_animation.gif", generate_tsne=True, generate_cm=True):
    results = {}
    for M in values_of_M:
        # Create model instance
        model = CNN(use_dropout = True, dropout_rate = 0.3)
        trainer = KMeansConsistencyTrainer(model, device=device)

        # Split dataset
        labeled_data, unlabeled_data = split_dataset(mnist_train, M)
        # print_label_distribution(labeled_data, description=f"Labeled Dataset for M={M}")
        
        labeled_loader = torch.utils.data.DataLoader(labeled_data, batch_size=256, shuffle=True, num_workers=0)
        unlabeled_loader = torch.utils.data.DataLoader(unlabeled_data, batch_size=256, shuffle=True, num_workers=0) if use_unlabeled else None

        # Train and log results
        train_accs, test_accs = trainer.train(
            labeled_loader=labeled_loader, 
            unlabeled_loader=unlabeled_loader, 
            test_loader=test_loader, 
            epochs=epochs, 
            evaluate_every=evaluate_every
        )
        results[M] = (train_accs, test_accs)
        # print(f"Train accuracy for M={M}: {train_accs[-1]:.2f}%, Test accuracy for M={M}: {test_accs[-1]:.2f}%")


        # Evaluate and compute confusion matrix
        test_accuracy, preds, labels = trainer.evaluate(test_loader)
        if generate_cm:
            cm = confusion_matrix(labels, preds)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
            disp.plot(cmap="viridis")
            plt.title(f"Confusion Matrix for M={M}")
            plt.show()

        if generate_tsne:
            # Plot t-SNE visualization
            plot_tsne(model, M, test_loader)

    if generate_tsne:
        create_animation(save_dir, output_file)
    return results

# Plot training curves
def plot_training_curves(results, values_of_M):
    plt.figure(figsize=(12, 6))
    
    # Plot test accuracy for each M
    for M in values_of_M:
        train_accs, test_accs = results[M]
        plt.plot(test_accs, label=f"M={M} (Test Accuracy)")

    plt.title("Test Accuracy vs. Epochs for Different Values of M")
    plt.xlabel("Epoch")
    plt.ylabel("Test Accuracy (%)")
    plt.ylim(0, 100)
    plt.legend()
    plt.grid()
    plt.show()

# Plot t-SNE visualization
def plot_tsne(model, M, data_loader, save_dir="tsne_images"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    model.eval()
    features_list = []
    labels_list = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            _, features = model(images)
            features_list.append(features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    features = np.concatenate(features_list)
    labels = np.concatenate(labels_list)
    tsne = TSNE(n_components=2, random_state=0)
    tsne_results = tsne.fit_transform(features)
    plt.figure(figsize=(6, 6))
    scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=labels, cmap='viridis', alpha=0.5)
    plt.colorbar(scatter)
    plt.title(f't-SNE Visualization for M={M}')
    plt.savefig(os.path.join(save_dir, f'tsne_M_{M}.png'))
    plt.show()  # Display the plot in the output
    plt.close()

def create_animation(save_dir="tsne_images", output_file="tsne_animation.gif"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    fig = plt.figure(figsize=(6, 6))
    images = []
    
    # Extract numeric values of M from filenames and sort them
    file_names = sorted(
        [f for f in os.listdir(save_dir) if f.startswith('tsne_M_') and f.endswith('.png')],
        key=lambda x: int(x.split('_')[2].split('.')[0])
    )
    
    for file_name in file_names:
        img = plt.imread(os.path.join(save_dir, file_name))
        images.append([plt.imshow(img, animated=True)])
    
    ani = animation.ArtistAnimation(fig, images, interval=500, blit=True, repeat_delay=1000)
    ani.save(os.path.join(save_dir, output_file), writer='imagemagick')
    plt.close()


<h1> Hyperparameter Optimization with Optuna </h1>

In [4]:
def objective(trial):
    # Define the hyperparameter search space for learning rates
    lr_simclr = trial.suggest_loguniform('lr_simclr', 1e-4, 1e-2)
    lr_finetune = trial.suggest_loguniform('lr_finetune', 1e-4, 1e-2)
    
    # Use a fixed value of M for the labeled dataset size
    M = 100  # You can adjust this value
    
    # Create model and trainer
    model = CNN(use_dropout=True, dropout_rate=0.3, use_projection_head=True)
    model = model.to(device)
    trainer = SimCLRTrainer(model, lr_simclr=lr_simclr, lr_finetune=lr_finetune)

    # Split dataset for SimCLR pretraining and supervised fine-tuning 
    split = [len(mnist_train) - M, M]
    simclr_dataset, finetune_dataset = random_split(mnist_train, split)
    
    # Create data loaders
    simclr_loader = DataLoader(simclr_dataset, batch_size=256, shuffle=True, num_workers=0)
    finetune_loader = DataLoader(finetune_dataset, batch_size=128, shuffle=True, num_workers=0)
    
    try:
        # Pretrain with SimCLR
        trainer.train(
            train_loader=simclr_loader,
            epochs=10  # You can adjust epochs
        )

        # Fine-tune on labeled data
        _, test_accs = trainer.fine_tune(
            train_loader=finetune_loader,
            test_loader=test_loader,
            epochs=10,  # You can adjust epochs
            evaluate_every=2
        )
        
        return max(test_accs)  # Return best test accuracy
        
    except Exception as e:
        print(f"Trial failed: {e}")
        return float('-inf')  # Return worst possible score on failure


In [ ]:
# Create an Optuna study and optimize the objective function
set_seed(7 * 5 * 3 * 2 * 2)

# Create and run the study
study_name = "Optimize_SimCLR_Learning_Rates"
study = optuna.create_study(study_name=study_name, direction='maximize')
study.optimize(objective, n_trials=50)  # Adjust number of trials as needed

# Print results
print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-01-02 20:10:58,681] A new study created in memory with name: Optimize_SimCLR_Learning_Rates


[2025-01-02 20:11:59] Epoch [1/10], Loss: 4.6124
[2025-01-02 20:13:04] Epoch [2/10], Loss: 4.4452
[2025-01-02 20:14:17] Epoch [3/10], Loss: 4.4229
[2025-01-02 20:15:17] Epoch [4/10], Loss: 4.4124
[2025-01-02 20:16:14] Epoch [5/10], Loss: 4.4037
[2025-01-02 20:17:12] Epoch [6/10], Loss: 4.3965
[2025-01-02 20:18:12] Epoch [7/10], Loss: 4.3906
[2025-01-02 20:19:20] Epoch [8/10], Loss: 4.3851
[2025-01-02 20:20:33] Epoch [9/10], Loss: 4.3819
[2025-01-02 20:21:48] Epoch [10/10], Loss: 4.3799
[2025-01-02 20:21:51] Accuracy: 19.14%
2025-01-02 20:21:49 Epoch [1/10], Loss: 2.7760, Train Accuracy: 9.00%, Test Accuracy: 19.14%
[2025-01-02 20:21:53] Accuracy: 36.31%
2025-01-02 20:21:51 Epoch [2/10], Loss: 2.1441, Train Accuracy: 20.00%, Test Accuracy: 36.31%
2025-01-02 20:21:53 Epoch [3/10], Loss: 1.8470, Train Accuracy: 37.00%, Test Accuracy: 36.31%
[2025-01-02 20:21:55] Accuracy: 53.16%
2025-01-02 20:21:53 Epoch [4/10], Loss: 1.6225, Train Accuracy: 51.00%, Test Accuracy: 53.16%
2025-01-02 20:21:

[I 2025-01-02 20:22:01,377] Trial 0 finished with value: 70.02 and parameters: {'lr_simclr': 0.0011709754734261458, 'lr_finetune': 0.002138136621969696}. Best is trial 0 with value: 70.02.


[2025-01-02 20:22:01] Accuracy: 70.02%
2025-01-02 20:21:59 Epoch [10/10], Loss: 0.5964, Train Accuracy: 77.00%, Test Accuracy: 70.02%
[2025-01-02 20:23:14] Epoch [1/10], Loss: 4.5872
[2025-01-02 20:24:29] Epoch [2/10], Loss: 4.4551
[2025-01-02 20:25:37] Epoch [3/10], Loss: 4.4311
[2025-01-02 20:26:44] Epoch [4/10], Loss: 4.4186
[2025-01-02 20:27:49] Epoch [5/10], Loss: 4.4101
[2025-01-02 20:29:05] Epoch [6/10], Loss: 4.4040
[2025-01-02 20:30:20] Epoch [7/10], Loss: 4.3993
[2025-01-02 20:31:33] Epoch [8/10], Loss: 4.3952
[2025-01-02 20:32:46] Epoch [9/10], Loss: 4.3927
[2025-01-02 20:33:55] Epoch [10/10], Loss: 4.3912
[2025-01-02 20:33:57] Accuracy: 41.80%
2025-01-02 20:33:55 Epoch [1/10], Loss: 2.5103, Train Accuracy: 12.00%, Test Accuracy: 41.80%
[2025-01-02 20:33:59] Accuracy: 48.12%
2025-01-02 20:33:57 Epoch [2/10], Loss: 1.9004, Train Accuracy: 54.00%, Test Accuracy: 48.12%
2025-01-02 20:33:59 Epoch [3/10], Loss: 1.4718, Train Accuracy: 65.00%, Test Accuracy: 48.12%
[2025-01-02 20:

[I 2025-01-02 20:34:07,652] Trial 1 finished with value: 81.27 and parameters: {'lr_simclr': 0.000936594021289934, 'lr_finetune': 0.006215873865120272}. Best is trial 1 with value: 81.27.


...

[2025-01-03 01:04:11] Accuracy: 81.22%
2025-01-03 01:04:10 Epoch [10/10], Loss: 0.2358, Train Accuracy: 92.00%, Test Accuracy: 81.22%
[2025-01-03 01:05:17] Epoch [1/10], Loss: 4.5951
[2025-01-03 01:06:23] Epoch [2/10], Loss: 4.4688
[2025-01-03 01:07:29] Epoch [3/10], Loss: 4.4450
[2025-01-03 01:08:38] Epoch [4/10], Loss: 4.4311
[2025-01-03 01:09:47] Epoch [5/10], Loss: 4.4217
[2025-01-03 01:10:58] Epoch [6/10], Loss: 4.4144
[2025-01-03 01:12:09] Epoch [7/10], Loss: 4.4081
[2025-01-03 01:13:26] Epoch [8/10], Loss: 4.4040
[2025-01-03 01:14:43] Epoch [9/10], Loss: 4.4013
[2025-01-03 01:16:04] Epoch [10/10], Loss: 4.3998
[2025-01-03 01:16:06] Accuracy: 31.10%
2025-01-03 01:16:04 Epoch [1/10], Loss: 2.3967, Train Accuracy: 13.00%, Test Accuracy: 31.10%
[2025-01-03 01:16:07] Accuracy: 44.72%
2025-01-03 01:16:06 Epoch [2/10], Loss: 1.8038, Train Accuracy: 41.00%, Test Accuracy: 44.72%
2025-01-03 01:16:08 Epoch [3/10], Loss: 1.3278, Train Accuracy: 58.00%, Test Accuracy: 44.72%
[2025-01-03 01:

[I 2025-01-03 01:16:15,431] Trial 27 finished with value: 78.59 and parameters: {'lr_simclr': 0.0006438465216914077, 'lr_finetune': 0.006326102212212433}. Best is trial 23 with value: 83.97.


[2025-01-03 01:16:15] Accuracy: 77.60%
2025-01-03 01:16:13 Epoch [10/10], Loss: 0.0709, Train Accuracy: 96.00%, Test Accuracy: 77.60%
[2025-01-03 01:17:34] Epoch [1/10], Loss: 4.6131
[2025-01-03 01:18:59] Epoch [2/10], Loss: 4.4795
[2025-01-03 01:20:23] Epoch [3/10], Loss: 4.4564
[2025-01-03 01:21:44] Epoch [4/10], Loss: 4.4419
[2025-01-03 01:23:07] Epoch [5/10], Loss: 4.4319
[2025-01-03 01:24:29] Epoch [6/10], Loss: 4.4249
[2025-01-03 01:25:47] Epoch [7/10], Loss: 4.4192
[2025-01-03 01:27:09] Epoch [8/10], Loss: 4.4160
[2025-01-03 01:28:27] Epoch [9/10], Loss: 4.4138
[2025-01-03 01:29:48] Epoch [10/10], Loss: 4.4130
[2025-01-03 01:29:50] Accuracy: 23.52%
2025-01-03 01:29:48 Epoch [1/10], Loss: 2.3289, Train Accuracy: 18.00%, Test Accuracy: 23.52%
[2025-01-03 01:29:53] Accuracy: 30.75%
2025-01-03 01:29:50 Epoch [2/10], Loss: 2.2085, Train Accuracy: 20.00%, Test Accuracy: 30.75%
2025-01-03 01:29:53 Epoch [3/10], Loss: 2.0272, Train Accuracy: 34.00%, Test Accuracy: 30.75%
[2025-01-03 01:

[I 2025-01-03 01:30:02,027] Trial 28 finished with value: 60.76 and parameters: {'lr_simclr': 0.0004923043264801173, 'lr_finetune': 0.00042942827936859844}. Best is trial 23 with value: 83.97.


[2025-01-03 01:30:02] Accuracy: 60.76%
2025-01-03 01:29:59 Epoch [10/10], Loss: 1.1836, Train Accuracy: 72.00%, Test Accuracy: 60.76%
[2025-01-03 01:31:21] Epoch [1/10], Loss: 4.5578
[2025-01-03 01:32:42] Epoch [2/10], Loss: 4.4496
[2025-01-03 01:34:02] Epoch [3/10], Loss: 4.4249
[2025-01-03 01:35:23] Epoch [4/10], Loss: 4.4109
[2025-01-03 01:36:45] Epoch [5/10], Loss: 4.4009
[2025-01-03 01:38:05] Epoch [6/10], Loss: 4.3940
[2025-01-03 01:39:20] Epoch [7/10], Loss: 4.3879
[2025-01-03 01:40:45] Epoch [8/10], Loss: 4.3835


<h1> Visualize Hyperparameter Optimization Results </h1>

In [ ]:
# Optimization History Plot
history_plot = optuna.visualization.plot_optimization_history(study)
history_plot.show()

# Hyperparameter Importance Plot
importance_plot = optuna.visualization.plot_param_importances(study)
importance_plot.show()

# Parallel Coordinate Plot
parallel_plot = optuna.visualization.plot_parallel_coordinate(study)
parallel_plot.show()

# Slice Plot
slice_plot = optuna.visualization.plot_slice(study)
slice_plot.show()

# Contour Plot
contour_plot = optuna.visualization.plot_contour(study)
contour_plot.show()

# EDF Plot
edf_plot = optuna.visualization.plot_edf(study)
edf_plot.show()